In [105]:
import altair as alt
import pandas as pd
pd.set_option('display.max_columns', None)

In [106]:
migration_df = pd.read_csv('migration_flow.csv')
migration_df.sample(5)

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,CITIZENSHIP,Citizenship,FREQ,Frequency of observation,MEASURE,Measure,SEX,Sex,BIRTH_PLACE,Place of birth,EDUCATION_LEV,Education level,UNIT_MEASURE,Unit of measure,TIME_PERIOD,Time period,OBS_VALUE,Observation value,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,DECIMALS,Decimals
198302,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,MEX,Mexico,NPL,Nepal,A,Annual,B11,Inflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2020.0,NaN,1.0,NaN,A,Normal value,0.0,Units,0,Zero
694,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,BEL,Belgium,SUR,Suriname,A,Annual,B12,Outflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2021.0,NaN,176.0,NaN,A,Normal value,0.0,Units,0,Zero
10556,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,FRA,France,BIH,Bosnia and Herzegovina,A,Annual,B11,Inflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2011.0,NaN,313.0,NaN,A,Normal value,0.0,Units,0,Zero
127886,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,AUS,Australia,CAF,Central African Republic,A,Annual,B11,Inflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2005.0,NaN,1.0,NaN,A,Normal value,0.0,Units,0,Zero
81886,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,NOR,Norway,TKL,Tokelau,A,Annual,B11,Inflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2018.0,NaN,0.0,NaN,A,Normal value,0.0,Units,0,Zero


# Data Cleanup

In [107]:

columns_to_drop = [
    "STRUCTURE", "STRUCTURE_ID", "STRUCTURE_NAME", "ACTION",
    "FREQ", "Frequency of observation", "SEX", "Sex",
    "BIRTH_PLACE", "Place of birth", "EDUCATION_LEV", "Education level",
    "OBS_STATUS", "Observation status", "UNIT_MULT", "Unit multiplier",
    "DECIMALS", "Decimals", "UNIT_MEASURE", "Unit of measure", "Time period", "Observation value"
]

df_cleaned = migration_df.drop(columns=columns_to_drop)


In [108]:
df_cleaned.sample(5)

,REF_AREA,Reference area,CITIZENSHIP,Citizenship,MEASURE,Measure,TIME_PERIOD,OBS_VALUE
2206,SVN,Slovenia,DZA,Algeria,B11,Inflows of foreign population,2010.0,3.0
75415,ITA,Italy,W,World,B12,Outflows of foreign population,2017.0,40551.0
172388,AUT,Austria,GRC,Greece,B11,Inflows of foreign population,2009.0,430.0
27079,ESP,Spain,MNG,Mongolia,B12,Outflows of foreign population,2005.0,1.0
142024,ISL,Iceland,MCO,Monaco,B12,Outflows of foreign population,2016.0,0.0


# Creating distinct datasets for inbound and outbound migration

In [109]:

inflow_df = df_cleaned[df_cleaned['Measure'] == 'Inflows of foreign population']
inflow_df.sample(5)

,REF_AREA,Reference area,CITIZENSHIP,Citizenship,MEASURE,Measure,TIME_PERIOD,OBS_VALUE
217068,AUT,Austria,CHL,Chile,B11,Inflows of foreign population,2010.0,75.0
12866,AUT,Austria,MNG,Mongolia,B11,Inflows of foreign population,2013.0,309.0
70918,DEU,Germany,NAM,Namibia,B11,Inflows of foreign population,2016.0,109.0
68834,AUT,Austria,BIH,Bosnia and Herzegovina,B11,Inflows of foreign population,2005.0,4626.0
202887,POL,Poland,BGD,Bangladesh,B11,Inflows of foreign population,2009.0,85.0


In [110]:
inflow_df = inflow_df.fillna(0)
inflow_df["TIME_PERIOD"] = inflow_df["TIME_PERIOD"].astype(int)

In [111]:
inflow_df = inflow_df[inflow_df['TIME_PERIOD'] != 0]

In [112]:
inflows_pivot = inflow_df.pivot_table(
    index=["Reference area", "REF_AREA"],
    columns="TIME_PERIOD",
    values="OBS_VALUE",
    aggfunc="sum")

In [113]:
inflows_pivot.columns = inflows_pivot.columns.astype(str)
inflows_pivot.fillna(0, inplace=True)

In [114]:
inflows_pivot.sample(5)

,TIME_PERIOD,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Reference area,REF_AREA,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Finland,FIN,13034.0,13556.0,17625.0,18180.0,17335.0,19896.0,23805.0,21629.0,20670.0,24885.0,27617.0,29892.0,37493.0,42324.0,38089.0,36115.0,40565.0,46668.0,47746.0,47294.0,42828.0,54548.001,47470.00000,46292.0,48289.067,46495.0,56085.0,84939.0
Sweden,SWE,69550.0,58442.0,64997.0,70628.0,69562.0,95507.0,99736.0,107186.0,107682.0,94987.0,102437.0,160299.0,166805.0,166290.0,166321.0,156766.0,150449.0,164223.0,189870.0,212028.0,227549.0,285798.000,249779.00000,228698.0,196360.000,131538.0,148304.0,172526.0
Luxembourg,LUX,23351.0,18372.0,18824.0,21079.0,23467.0,30039.0,30674.0,30085.0,34240.0,33334.0,37279.0,37820.0,31532.0,33483.0,29139.0,31429.0,38022.0,38659.0,39409.0,41916.0,45061.0,42973.000,46280.00000,46632.0,50236.000,41903.0,47445.0,59610.0
Estonia,EST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1473.0,1910.0,2934.0,3857.0,3858.0,4423.0,2367.0,3309.0,2185.0,3230.0,2690.0,14737.0,14995.001,17703.00001,18935.0,21579.000,15752.0,19190.0,84603.0
Japan,JPN,365075.0,391964.0,486794.0,465954.0,501739.0,622845.0,632640.0,620688.0,674774.0,672531.0,662459.0,674410.0,695867.0,712713.0,615881.0,574108.0,533820.0,607610.0,613300.0,672873.0,782077.0,854861.000,949602.00000,1038983.0,1183504.000,441018.0,159845.0,1122992.0


In [115]:
outflow_df = df_cleaned[df_cleaned['Measure'] == 'Outflows of foreign population']
outflow_df.sample(5)

,REF_AREA,Reference area,CITIZENSHIP,Citizenship,MEASURE,Measure,TIME_PERIOD,OBS_VALUE
124927,SVN,Slovenia,ITA,Italy,B12,Outflows of foreign population,2020.0,402.0
31880,FIN,Finland,STLS,Stateless,B12,Outflows of foreign population,2003.0,4.0
72659,HUN,Hungary,UKR,Ukraine,B12,Outflows of foreign population,2016.0,439.0
111228,AUS,Australia,MDG,Madagascar,B12,Outflows of foreign population,1997.0,1.0
186729,CZE,Czechia,MLT,Malta,B12,Outflows of foreign population,2005.0,1.0


In [116]:
outflow_df = outflow_df.fillna(0)
outflow_df["TIME_PERIOD"] = outflow_df["TIME_PERIOD"].astype(int)

In [117]:
outflows_pivot = outflow_df.pivot_table(
    index=["Reference area", "REF_AREA"],
    columns="TIME_PERIOD",
    values="OBS_VALUE",
    aggfunc="sum"
)

In [118]:
outflows_pivot.reset_index(inplace=True)
outflows_pivot.fillna(0, inplace=True)

In [119]:
outflows_pivot.sample(5)

TIME_PERIOD,Reference area,REF_AREA,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
7,France,FRA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23817.0,8086.0,37908.0,40948.0,38370.0,49296.0,51923.0,50180.0,21547.0,30067.0,29910.0,21720.0,30348.0,0.0
16,Latvia,LVA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6681.0,4742.0,3411.0,1391.0,2563.0,3041.0,2318.0,2889.0,8171.0,7467.0,7427.0,0.0
0,Australia,AUS,31058.0,35299.0,36295.0,16298.0,17947.0,19849.0,22978.0,1638.0,1608.0,59278.0,57205.0,59975.0,61564.0,64063.0,57608.0,61168.0,62332.0,59566.0,63291.0,65151.0,66900.0,65400.0,0.0,0.0,0.0,0.0,0.0,0.0
28,Switzerland,CHE,157198.0,153450.0,150856.0,138433.0,137958.0,130882.0,122576.0,115260.0,107056.0,111014.0,116208.0,124850.0,134832.0,125089.0,129724.0,130913.0,127844.0,131499.0,139792.0,138075.0,146689.0,154950.0,157979.0,161289.0,160039.0,140311.0,148521.0,0.0
18,Luxembourg,LUX,8832.0,11120.0,11541.0,13420.0,13776.0,19575.0,21418.0,22991.0,19157.0,20704.0,20166.0,21641.0,17282.0,15909.0,14596.0,15195.0,14875.0,17105.0,17771.0,18812.0,20747.0,22558.0,23043.0,23248.0,25983.0,24062.0,25771.0,0.0


In [120]:
from vega_datasets import data

world = data.world_110m.url

In [121]:
countries_df = pd.read_csv("countries.csv")
countries_df.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [122]:
iso_url = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv'
iso_df = pd.read_csv(iso_url)

iso_df.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [123]:
# add country-code to outflows_pivot from iso_df

outflows_pivot = outflows_pivot.merge(iso_df[["country-code", "alpha-3"]], left_on='REF_AREA', right_on='alpha-3', how='left')

outflows_pivot.sample(5)

,Reference area,REF_AREA,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,country-code,alpha-3
20,New Zealand,NZL,13162.0,15417.0,17697.0,19316.0,19057.0,31248.0,54624.0,54502.0,68121.0,76733.0,76139.0,73266.0,74257.0,79184.0,82414.0,87594.0,90324.0,82947.0,74620.0,71412.0,73987.0,78339.0,90618.0,94130.0,110111.0,78606.0,88065.0,87423.0,554,NZL
4,Denmark,DNK,11337.0,12009.0,24355.0,31643.0,33948.0,33625.0,35311.0,35289.0,37121.0,36932.0,37953.0,40050.0,43727.0,53033.0,53170.0,54155.0,53133.0,58183.0,59452.0,60828.0,61186.0,74689.0,83027.0,90602.0,104531.0,83376.0,62868.0,0.0,208,DNK
0,Australia,AUS,31058.0,35299.0,36295.0,16298.0,17947.0,19849.0,22978.0,1638.0,1608.0,59278.0,57205.0,59975.0,61564.0,64063.0,57608.0,61168.0,62332.0,59566.0,63291.0,65151.0,66900.0,65400.0,0.0,0.0,0.0,0.0,0.0,0.0,36,AUS
26,Spain,ESP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15986.0,38520.0,90187.0,103926.0,244549.0,409036.0,509784.0,688203.0,726374.0,706950.0,778539.0,917722.0,639823.0,498371.0,474890.0,563818.0,460355.0,439735.0,354246.0,587297.0,0.0,724,ESP
29,Türkiye,TUR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,283756.0,391057.0,508180.0,488554.0,681499.0,367075.0,0.0,792,TUR


In [124]:
outflows_pivot["country-code"] = outflows_pivot["country-code"].astype(int)

In [125]:
outflows_pivot.drop(columns=["alpha-3"], inplace=True)

In [126]:
outflows_pivot.columns = outflows_pivot.columns.astype(str)


In [ ]:
world = alt.topo_feature(data.world_110m.url, "countries")

base_map = alt.Chart(world).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project(
    type='naturalEarth1'
).properties(
    width=800,
    height=400
)


choropleth = (
    alt.Chart(world)
    .mark_geoshape().encode(
        color=alt.Color(
            "1995:Q",
            scale=alt.Scale(scheme="lightgreyred"),
            legend=alt.Legend(title="Migration Flow"),
        ),
        tooltip=[
            alt.Tooltip("Reference area:N", title="Country"),
            alt.Tooltip("1995:Q", title="Migration Flow"), # need a slider now
        ],
    )
    .transform_lookup(
        lookup="id", from_=alt.LookupData(outflows_pivot, "country-code", ["1995", "Reference area"])
    )
    .properties(width=800, height=400)
    .project("naturalEarth1")
)


base_map + choropleth

alt.LayerChart(...)

In [128]:
inflows_pivot.sample(5)

,TIME_PERIOD,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Reference area,REF_AREA,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Greece,GRC,0.0,0.0,0.0,71226.0,0.0,0.0,0.0,0.0,0.0,0.0,118365.0,115858.0,87105.0,80404.0,87353.0,62276.0,42541.0,17732.000000,31302.000000,29510.000000,33986.000000,86120.000000,80504.0,87290.0,95385.0,63357.0,28728.0,62640.0
Japan,JPN,365075.0,391964.0,486794.0,465954.0,501739.0,622845.0,632640.0,620688.0,674774.0,672531.0,662459.0,674410.0,695867.0,712713.0,615881.0,574108.0,533820.0,607610.000000,613300.000000,672873.000000,782077.000000,854861.000000,949602.0,1038983.0,1183504.0,441018.0,159845.0,1122992.0
Spain,ESP,19538.0,16686.0,71197.0,136560.0,230337.0,702191.0,838155.0,952155.0,928897.0,1382277.0,1465780.0,1717791.0,1960462.0,1134638.0,730631.0,660420.0,671519.0,544681.952235,496405.044579,528630.757128,579483.464074,703712.318332,907386.0,1119351.0,1331357.0,805654.0,1455643.0,2181494.0
Sweden,SWE,69550.0,58442.0,64997.0,70628.0,69562.0,95507.0,99736.0,107186.0,107682.0,94987.0,102437.0,160299.0,166805.0,166290.0,166321.0,156766.0,150449.0,164223.000000,189870.000000,212028.000000,227549.000000,285798.000000,249779.0,228698.0,196360.0,131538.0,148304.0,172526.0
Switzerland,CHE,193016.0,169895.0,157543.0,160572.0,182489.0,187748.0,209902.0,218714.0,209009.0,220508.0,219231.0,240741.0,336245.0,383246.0,310220.0,265939.0,282314.0,285127.000000,308156.000000,301872.000000,298127.000000,283412.000000,272960.0,277443.0,279217.0,271856.0,280609.0,322521.0


In [129]:
inflows_pivot.reset_index(inplace=True)
inflows_pivot.fillna(0, inplace=True)
inflows_pivot = inflows_pivot.merge(iso_df[["country-code", "alpha-3"]], left_on='REF_AREA', right_on='alpha-3', how='left')
inflows_pivot["country-code"] = inflows_pivot["country-code"].astype(int)
inflows_pivot.drop(columns=["alpha-3"], inplace=True)
inflows_pivot.columns = inflows_pivot.columns.astype(str)
inflows_pivot.sample(5)

,Reference area,REF_AREA,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,country-code
10,Finland,FIN,13034.0,13556.0,17625.0,18180.0,17335.0,19896.0,23805.0,21629.0,20670.0,24885.0,27617.0,29892.0,37493.0,42324.0,38089.0,36115.0,40565.0,46668.0,47746.0,47294.0,42828.0,54548.001,47470.0,46292.0,48289.067,46495.0,56085.0,84939.0,246
35,Türkiye,TUR,192252.0,247872.0,309927.0,346215.0,64466.0,69137.0,66425.0,67520.0,70905.0,73435.0,84917.0,104023.0,0.0,33562.0,764.0,59494.0,0.0,0.0,0.0,0.0,0.0,544433.000,725398.0,930763.0,1153793.000,482389.0,1227522.0,796057.0,792
20,Korea,KOR,0.0,0.0,0.0,0.0,0.0,346134.0,327783.0,317871.0,337745.0,357050.0,507432.0,605909.0,600708.0,604341.0,465676.0,586130.0,614481.0,600344.0,720939.0,814111.0,745864.0,804400.000,905305.0,990140.0,876405.000,466230.0,441117.0,825861.0,410
14,Hungary,HUN,28546.0,28318.0,27472.0,32751.0,41112.0,41157.0,41242.0,36135.0,38969.0,43124.0,58867.0,1609064.0,46520.0,77020.0,56549.0,47596.0,44819.0,40416.0,42246.0,51781.0,51350.0,47429.000,72592.0,98236.0,110150.000,87274.0,97779.0,110668.0,348
23,Luxembourg,LUX,23351.0,18372.0,18824.0,21079.0,23467.0,30039.0,30674.0,30085.0,34240.0,33334.0,37279.0,37820.0,31532.0,33483.0,29139.0,31429.0,38022.0,38659.0,39409.0,41916.0,45061.0,42973.000,46280.0,46632.0,50236.000,41903.0,47445.0,59610.0,442


In [ ]:
in_choropleth = (
    alt.Chart(world)
    .mark_geoshape().encode(
        color=alt.Color(
            "1995:Q",
            scale=alt.Scale(scheme="blues"),
            legend=alt.Legend(title="Migration Flow"),
        ),
        tooltip=[
            alt.Tooltip("Reference area:N", title="Country"),
            alt.Tooltip("1995:Q", title="Migration Flow"), # need a slider now
        ],
    )
    .transform_lookup(
        lookup="id", from_=alt.LookupData(inflows_pivot, "country-code", ["1995", "Reference area"])
    )
    .properties(width=800, height=400)
    .project("naturalEarth1")
)

base_map + in_choropleth

alt.LayerChart(...)